**I - Import the librairies and modules**

In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

pd.set_option('display.max_columns', None)

**II - Import the data** : data set is df

In [13]:
df = pd.read_csv("../raw_data/FC26_20250921.csv", low_memory=False)

In [31]:
df.head(5)

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,player_face_url
0,252371,/player/252371/jude-bellingham/260004/,26,4,2025-09-19,J. Bellingham,Jude Victor William Bellingham,"CAM, CM",90,94,174500000,320000,22,2003-06-29,186,75,53.0,La Liga,1.0,243.0,Real Madrid,CAM,5.0,NaN,2023-07-01,2029.0,14,England,1318.0,CAM,10.0,Right,4,4,5,Normal (170-185),Yes,370800000.0,"#Dribbler, #Playmaker , #Clinical finisher, #C...","Relentless +, Low Driven Shot, Gamechanger, In...",80.0,86.0,83.0,90.0,78.0,85.0,66,88,75,90,77,91,73,68,https://cdn.sofifa.net/players/252/371/26_120.png
1,239053,/player/239053/federico-valverde/260004/,26,4,2025-09-19,F. Valverde,Federico Santiago Valverde Dipetta,"CM, CDM, RB",89,90,120500000,340000,26,1998-07-22,182,74,53.0,La Liga,1.0,243.0,Real Madrid,RDM,8.0,NaN,2016-07-22,2029.0,60,Uruguay,NaN,NaN,NaN,Right,4,3,4,Unique,Yes,256100000.0,"#Playmaker , #Distance shooter, #Tackling , #T...","Relentless +, Power Shot, Long Ball Pass, Inte...",88.0,84.0,84.0,84.0,83.0,85.0,78,80,63,88,78,84,76,69,https://cdn.sofifa.net/players/239/053/26_120.png
2,212622,/player/212622/joshua-kimmich/260004/,26,4,2025-09-19,J. Kimmich,Joshua Walter Kimmich,"CDM, RB, CM",89,89,86000000,140000,30,1995-02-08,177,75,19.0,Bundesliga,1.0,21.0,FC Bayern München,RDM,6.0,NaN,2015-07-01,2029.0,21,Germany,1337.0,RCM,6.0,Right,4,3,4,Normal (170-185),Yes,141900000.0,"#Playmaker , #Crosser, #Tackling , #Complete m...","Long Ball Pass +, Finesse Shot, Whipped Pass, ...",72.0,74.0,89.0,84.0,83.0,79.0,92,68,72,89,69,82,85,79,https://cdn.sofifa.net/players/212/622/26_120.png
3,235212,/player/235212/achraf-hakimi/260004/,26,4,2025-09-19,A. Hakimi,Achraf Hakimi Mouhأشرف حكيمي,"RB, RM",89,90,111000000,170000,26,1998-11-04,181,73,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,RB,2.0,NaN,2021-07-06,2029.0,129,Morocco,111111.0,RB,2.0,Right,4,4,4,Lean (170-185),Yes,213700000.0,#Speedster,"Relentless +, Low Driven Shot, Whipped Pass, J...",92.0,79.0,82.0,83.0,82.0,79.0,85,80,71,86,74,82,78,80,https://cdn.sofifa.net/players/235/212/26_120.png
4,224232,/player/224232/nicolo-barella/260004/,26,4,2025-09-19,N. Barella,Nicolò Barella,CM,87,87,79500000,69000,28,1997-02-07,175,68,31.0,Serie A,1.0,44.0,Inter,RCM,23.0,NaN,2020-09-01,2029.0,27,Italy,1343.0,RCM,18.0,Right,4,3,4,Normal (170-),Yes,131199999.0,#Acrobat,"Relentless +, Incisive Pass, Pinged Pass, Long...",80.0,78.0,84.0,86.0,81.0,76.0,79,77,60,89,80,84,79,67,https://cdn.sofifa.net/players/224/232/26_120.png


**III - Clean the data manually, start the analysis**

In [28]:
# Visualise our feature in a list : Numerical and Categorical features
features_columns = df.columns
features_columns

Index(['player_id', 'player_url', 'fifa_version', 'fifa_update',
       'fifa_update_date', 'short_name', 'long_name', 'player_positions',
       'overall', 'potential', 'value_eur', 'wage_eur', 'age', 'dob',
       'height_cm', 'weight_kg', 'league_id', 'league_name', 'league_level',
       'club_team_id', 'club_name', 'club_position', 'club_jersey_number',
       'club_loaned_from', 'club_joined_date',
       'club_contract_valid_until_year', 'nationality_id', 'nationality_name',
       'nation_team_id', 'nation_position', 'nation_jersey_number',
       'preferred_foot', 'weak_foot', 'skill_moves',
       'international_reputation', 'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'player_traits', 'pace',
       'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_a

In [29]:
# Additional profiling features used for filtering down the line
filtered_columns = ['player_id', 'player_url', 'short_name', 'long_name',
       'value_eur', 'wage_eur', 'age', 'dob', 'league_id', 'league_name',
       'league_level', 'club_team_id', 'club_name', 'club_position',
       'club_jersey_number', 'club_loaned_from', 'club_joined_date',
       'club_contract_valid_until_year', 'nationality_id', 'nationality_name',
       'nation_team_id', 'nation_position', 'nation_jersey_number',
       'body_type', 'real_face',
       'release_clause_eur', 'player_tags', 'player_traits',
       'player_face_url']

In [30]:
# Create features and info dataframes
player_features_df = df[features_columns]
player_info_df =  df[filtered_columns]

In [26]:
df

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,player_face_url
0,252371,/player/252371/jude-bellingham/260004/,26,4,2025-09-19,J. Bellingham,Jude Victor William Bellingham,"CAM, CM",90,94,174500000,320000,22,2003-06-29,186,75,53.0,La Liga,1.0,243.0,Real Madrid,CAM,5.0,NaN,2023-07-01,2029.0,14,England,1318.0,CAM,10.0,Right,4,4,5,Normal (170-185),Yes,370800000.0,"#Dribbler, #Playmaker , #Clinical finisher, #C...","Relentless +, Low Driven Shot, Gamechanger, In...",80.0,86.0,83.0,90.0,78.0,85.0,66,88,75,90,77,91,73,68,https://cdn.sofifa.net/players/252/371/26_120.png
1,239053,/player/239053/federico-valverde/260004/,26,4,2025-09-19,F. Valverde,Federico Santiago Valverde Dipetta,"CM, CDM, RB",89,90,120500000,340000,26,1998-07-22,182,74,53.0,La Liga,1.0,243.0,Real Madrid,RDM,8.0,NaN,2016-07-22,2029.0,60,Uruguay,NaN,NaN,NaN,Right,4,3,4,Unique,Yes,256100000.0,"#Playmaker , #Distance shooter, #Tackling , #T...","Relentless +, Power Shot, Long Ball Pass, Inte...",88.0,84.0,84.0,84.0,83.0,85.0,78,80,63,88,78,84,76,69,https://cdn.sofifa.net/players/239/053/26_120.png
2,212622,/player/212622/joshua-kimmich/260004/,26,4,2025-09-19,J. Kimmich,Joshua Walter Kimmich,"CDM, RB, CM",89,89,86000000,140000,30,1995-02-08,177,75,19.0,Bundesliga,1.0,21.0,FC Bayern München,RDM,6.0,NaN,2015-07-01,2029.0,21,Germany,1337.0,RCM,6.0,Right,4,3,4,Normal (170-185),Yes,141900000.0,"#Playmaker , #Crosser, #Tackling , #Complete m...","Long Ball Pass +, Finesse Shot, Whipped Pass, ...",72.0,74.0,89.0,84.0,83.0,79.0,92,68,72,89,69,82,85,79,https://cdn.sofifa.net/players/212/622/26_120.png
3,235212,/player/235212/achraf-hakimi/260004/,26,4,2025-09-19,A. Hakimi,Achraf Hakimi Mouhأشرف حكيمي,"RB, RM",89,90,111000000,170000,26,1998-11-04,181,73,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,RB,2.0,NaN,2021-07-06,2029.0,129,Morocco,111111.0,RB,2.0,Right,4,4,4,Lean (170-185),Yes,213700000.0,#Speedster,"Relentless +, Low Driven Shot, Whipped Pass, J...",92.0,79.0,82.0,83.0,82.0,79.0,85,80,71,86,74,82,78,80,https://cdn.sofifa.net/players/235/212/26_120.png
4,224232,/player/224232/nicolo-barella/260004/,26,4,2025-09-19,N. Barella,Nicolò Barella,CM,87,87,79500000,69000,28,1997-02-07,175,68,31.0,Serie A,1.0,44.0,Inter,RCM,23.0,NaN,2020-09-01,2029.0,27,Italy,1343.0,RCM,18.0,Right,4,3,4,Normal (170-),Yes,131199999.0,#Acrobat,"Relentless +, Incisive Pass, Pinged Pass, Long...",80.0,78.0,84.0,86.0,81.0,76.0,79,77,60,89,80,84,79,67,https://cdn.sofifa.net/players/224/232/26_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18400,267946,/player/267946/jun-sub-lim/260004/,26,4,2025-09-19,Lim Jun Sub,Joon-seob Im임준섭,GK,48,54,60000,500,21,2003-08-22,194,90,83.0,K League 1,1.0,982.0,FC Seoul,RES,25.0,NaN,2024-12-31,2030.0,167,Korea Republic,NaN,NaN,NaN,Right,2,1,1,Lean (185+),No,87000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,8,12,23,8,10,9,10,https://cdn.sofifa.net/players/267/946/26_120.png
18401,76593,/player/76593/jianxian-yu/260004/,26,4,2025-09-19,Yu Jianxian,Jianxian Yu,GK,49,56,80000,500,23,2001-11-06,188,77,2012.0,Super League,1.0,131531.0,Yunnan Yukun,SUB,24.0,NaN,2023-04-07,2027.0,155,China PR,NaN,NaN,NaN,Right,2,1,1,Normal (185+)

In [40]:
# Convert player position string into list of positions
player_features_df['player_positions'].apply(lambda x: x.split(', '))

# Take the first given position as a player's primary position (new column)
player_features_df['primary_position'] = player_features_df['player_positions'].apply(lambda x : x[0])

/tmp/ipykernel_46207/654346671.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_features_df['primary_position'] = player_features_df['player_positions'].apply(lambda x : x[0])


In [ ]:
numeric_columns = ['overall', 'potential', 'height_cm', 'weight_kg', 'weak_foot', 'skill_moves',
                'international_reputation', 'pace', 'shooting', 'passing', 'dribbling', 'defending',
                'physic', 'attacking_crossing', 'attacking_finishing','attacking_heading_accuracy',
                'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve',
                'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
                'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
                'movement_reactions', 'movement_balance', 'power_shot_power',
                'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
                'mentality_aggression', 'mentality_interceptions',
                'mentality_positioning', 'mentality_vision', 'mentality_penalties',
                'mentality_composure', 'defending_marking_awareness',
                'defending_standing_tackle', 'defending_sliding_tackle',
                'goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']

categorical_columns = ['preferred_foot', 'player_positions', 'primary_position']

In [44]:
# Fill in NaNs

# 0 for goalkeeping speed for all outfield players
player_features_df['goalkeeping_speed'] = player_features_df['goalkeeping_speed'].fillna(0)

# 0 for grouped outfield attribute scores for goalkeepers
columns_to_fill = ['pace', 'physic', 'defending', 'passing', 'shooting', 'dribbling']
player_features_df[columns_to_fill] = player_features_df[columns_to_fill].fillna(0)

In [45]:
player_features_df

,player_id,overall,potential,height_cm,weight_kg,weak_foot,skill_moves,international_reputation,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,primary_position_C,primary_position_G,primary_position_L,primary_position_R,primary_position_S,preferred_foot_Left,preferred_foot_Right
0,252371,0.977273,0.978261,0.563636,0.482759,0.75,0.75,1.00,0.746269,0.915493,0.865672,0.957746,0.840000,0.898305,0.689655,0.933333,0.793103,0.962963,0.831461,0.966292,0.767442,0.701149,0.952381,0.964706,0.804878,0.797619,0.851351,0.953125,0.840000,0.891892,0.852941,0.987952,0.788732,0.954023,0.891566,0.894118,0.956522,0.9750,0.784091,0.961538,0.837209,0.857143,0.855422,0.136364,0.102273,0.089888,0.034091,0.068182,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,239053,0.954545,0.891304,0.490909,0.465517,0.75,0.50,0.75,0.865672,0.887324,0.880597,0.873239,0.906667,0.898305,0.827586,0.844444,0.655172,0.938272,0.842697,0.887640,0.802326,0.712644,0.940476,0.929412,0.841463,0.940476,0.770270,0.921875,0.640000,0.959459,0.808824,0.975904,0.816901,1.000000,0.843373,0.941176,0.902174,0.9250,0.613636,0.884615,0.895349,0.952381,0.963855,0.045455,0.090909,0.044944,0.147727,0.068182,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,212622,0.954545,0.869565,0.400000,0.482759,0.75,0.50,0.75,0.626866,0.746479,0.955224,0.873239,0.906667,0.796610,0.988506,0.711111,0.758621,0.950617,0.741573,0.865169,0.906977,0.827586,0.988095,0.905882,0.731707,0.678571,0.851351,0.921875,0.853333,0.783784,0.735294,0.963855,0.633803,0.908046,0.939759,0.929412,0.815217,0.9250,0.750000,0.910256,0.895349,0.940476,0.951807,0.068182,0.147727,0.056180,0.147727,0.147727,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,235212,0.954545,0.891304,0.472727,0.448276,0.75,0.75,0.75,0.925373,0.816901,0.850746,0.859155,0.893333,0.796610,0.908046,0.844444,0.747126,0.913580,0.797753,0.865169,0.825581,0.839080,0.761905,0.894118,0.902439,0.976190,0.824324,0.937500,0.773333,0.837838,0.838235,1.000000,0.676056,0.839080,0.795181,0.929412,0.880435,0.8250,0.715909,0.884615,0.895349,0.928571,0.879518,0.090909,0.068182,0.134831,0.045455,0.068182,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,224232,0.909091,0.826087,0.363636,0.362069,0.75,0.50,0.75,0.746269,0.802817,0.880597,0.901408,0.880000,0.745763,0.839080,0.811111,0.620690,0.950617,0.865169,0.887640,0.837209,0.689655,0.928571,0.917647,0.817073,0.773810,0.959459,0.953125,0.906667,0.810811,0.705882,0.927711,0.591549,0.862069,0.915663,0.929412,0.880435,0.9125,0.772727,0.935897,0.872093,0.928571,0.891566,0.102273,0.034091,0.123596,0.136364,0.102273,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18400,267946,0.022727,0.108696,0.709091,0.741379,0.25,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.044444,0.068966,0.135802,0.056180,0.056180,0.023256,0.034483,0.154762,0.082353,0.134146,0.059524,0.040541,0.078125,0.026667,0.256757,0.235294,0.048193,0.408451,0.068966,0.084337,0.058824,0.076087,0.2250,0.056818,0.217949,0.081395,0.035714,0.084337,0.488636,0.556818,0.561798,0.500000,0.568182,0.16,0.0,1.0,0.0,0.0,0.0,0.0,1.0
18401,76593,0.045455,0.152174,0.600000,0.517241,0.25,0.00,0.00,0.000000,0.000000,0.000000

**IV - Scale the data : numerical and categories - VERIFIER LE OHE**

In [50]:
# Assign features df to X, with player ID as the index
X = player_features_df.copy()
X.set_index('player_id', inplace=True)

In [51]:
X

,overall,potential,height_cm,weight_kg,weak_foot,skill_moves,international_reputation,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,primary_position_C,primary_position_G,primary_position_L,primary_position_R,primary_position_S,preferred_foot_Left,preferred_foot_Right
player_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
252371,0.977273,0.978261,0.563636,0.482759,0.75,0.75,1.00,0.746269,0.915493,0.865672,0.957746,0.840000,0.898305,0.689655,0.933333,0.793103,0.962963,0.831461,0.966292,0.767442,0.701149,0.952381,0.964706,0.804878,0.797619,0.851351,0.953125,0.840000,0.891892,0.852941,0.987952,0.788732,0.954023,0.891566,0.894118,0.956522,0.9750,0.784091,0.961538,0.837209,0.857143,0.855422,0.136364,0.102273,0.089888,0.034091,0.068182,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
239053,0.954545,0.891304,0.490909,0.465517,0.75,0.50,0.75,0.865672,0.887324,0.880597,0.873239,0.906667,0.898305,0.827586,0.844444,0.655172,0.938272,0.842697,0.887640,0.802326,0.712644,0.940476,0.929412,0.841463,0.940476,0.770270,0.921875,0.640000,0.959459,0.808824,0.975904,0.816901,1.000000,0.843373,0.941176,0.902174,0.9250,0.613636,0.884615,0.895349,0.952381,0.963855,0.045455,0.090909,0.044944,0.147727,0.068182,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
212622,0.954545,0.869565,0.400000,0.482759,0.75,0.50,0.75,0.626866,0.746479,0.955224,0.873239,0.906667,0.796610,0.988506,0.711111,0.758621,0.950617,0.741573,0.865169,0.906977,0.827586,0.988095,0.905882,0.731707,0.678571,0.851351,0.921875,0.853333,0.783784,0.735294,0.963855,0.633803,0.908046,0.939759,0.929412,0.815217,0.9250,0.750000,0.910256,0.895349,0.940476,0.951807,0.068182,0.147727,0.056180,0.147727,0.147727,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
235212,0.954545,0.891304,0.472727,0.448276,0.75,0.75,0.75,0.925373,0.816901,0.850746,0.859155,0.893333,0.796610,0.908046,0.844444,0.747126,0.913580,0.797753,0.865169,0.825581,0.839080,0.761905,0.894118,0.902439,0.976190,0.824324,0.937500,0.773333,0.837838,0.838235,1.000000,0.676056,0.839080,0.795181,0.929412,0.880435,0.8250,0.715909,0.884615,0.895349,0.928571,0.879518,0.090909,0.068182,0.134831,0.045455,0.068182,0.00,0.0,0.0,0.0,1.0,0.0,0.0,1.0
224232,0.909091,0.826087,0.363636,0.362069,0.75,0.50,0.75,0.746269,0.802817,0.880597,0.901408,0.880000,0.745763,0.839080,0.811111,0.620690,0.950617,0.865169,0.887640,0.837209,0.689655,0.928571,0.917647,0.817073,0.773810,0.959459,0.953125,0.906667,0.810811,0.705882,0.927711,0.591549,0.862069,0.915663,0.929412,0.880435,0.9125,0.772727,0.935897,0.872093,0.928571,0.891566,0.102273,0.034091,0.123596,0.136364,0.102273,0.00,1.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267946,0.022727,0.108696,0.709091,0.741379,0.25,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.068966,0.044444,0.068966,0.135802,0.056180,0.056180,0.023256,0.034483,0.154762,0.082353,0.134146,0.059524,0.040541,0.078125,0.026667,0.256757,0.235294,0.048193,0.408451,0.068966,0.084337,0.058824,0.076087,0.2250,0.056818,0.217949,0.081395,0.035714,0.084337,0.488636,0.556818,0.561798,0.500000,0.568182,0.16,0.0,1.0,0.0,0.0,0.0,0.0,1.0
76593,0.045455,0.152174,0.600000,0.517241,0.25,0.00,0.

In [42]:
# MinMax scale numerical features
mm_scaler = MinMaxScaler()
player_features_df[numeric_columns] = mm_scaler.fit_transform(player_features_df[numeric_columns])

/tmp/ipykernel_46207/1480969605.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_features_df[numeric_columns] = mm_scaler.fit_transform(player_features_df[numeric_columns])


In [ ]:
# Positional groups and longer positional names if needed

# Map primary position to broader groups
position_groups = {
    'ST': 'Forward', 'CF': 'Forward', 'LW': 'Forward', 'RW': 'Forward',
    'CAM': 'Midfielder', 'CM': 'Midfielder', 'CDM': 'Midfielder', 'LM': 'Midfielder', 'RM': 'Midfielder',
    'CB': 'Defender', 'LB': 'Defender', 'RB': 'Defender', 'LWB': 'Defender', 'RWB': 'Defender',
    'GK': 'Goalkeeper'
}
# player_features_df['position_group'] = player_features_df['primary_position'].map(position_groups).fillna('Other')

# Dictionary matching long position names to short
{'CM': 'Central Midfielder',
'CB': 'Central Defender',
'RM': 'Right Midfielder',
'LM': 'Left Midfielder',
'ST': 'Striker',
'CAM': 'Central Attacking Midfielder',
'CDM': 'Central Defensive Midfielder',
'RB': 'Right Fullback',
'LB': 'Left Fullback',
'GK': 'Goalkeeper',
'LW': 'Left Winger',
'RW': 'Right Winger'}

**V - Pipeline for preprocessing the data set**

In [1]:
#import sklearn and module
from sklearn.pipeline import Pipeline

**One Hot Encode categorical features is a 1st solution to scale our categorical data btwn O and 1OHE player primary positions**

In [ ]:
categorical_columns=
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(player_features_df[['primary_position']])
#player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['primary_position']])

# OHE player prefered foot
#ohe = OneHotEncoder(sparse_output=False)
#ohe.fit(player_features_df[['preferred_foot']])
#player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['preferred_foot']])

# Drop the original categorical columns
#player_features_df = player_features_df.drop(columns=categorical_columns)


/tmp/ipykernel_46207/1210962741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['primary_position']])
/tmp/ipykernel_46207/1210962741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_features_df[ohe.get_feature_names_out()] = ohe.transform(player_features_df[['primary_position']])
/tmp/ipykernel_46207/1210962741.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

**However a 2nd solution would be to use a Random Forest Classifier - The interest would be to clean the dataset, preprocess in a single compute the data and classify without label**

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
#pipeline = Pipeline ([
    #("Cleaner", type_cleaner),
    #("Preprocessing", preproc),
    #("Classifier", RandomForestClassifier())])

**A 3rd solution would be to use a KNN on our categories. We can rely on the OHE realised yesterday**


In [25]:
# import pipeline ok
# import Simple Imputer : to fill any missing data although we should have a clean dataset this is for security
from sklearn.impute import SimpleImputer
# import Standard Scalers to harmonize our data. I choose to use the 3 of them for later accuracy evaluation
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

In [ ]:
#I choose my data now the method a pipeline with Simple imputer and OHE
Cat_preproc= Pipeline([
    ("cat_imputer",SimpleImputer(strategy=constant", fill_values="Missing"),
        ("ohe", OneHotEncoder(handle_unknown="ignore", sparse_output=False)
        